In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://www.1aauto.com/catalog/product/fit?skuId=' + input_.loc[a, 'Product_Id'] + '&pathName=' + input_.loc[a, 'Url'].replace('https://www.1aauto.com', '').replace('/', '%2F'),
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//tr[@class="sortable_row filter-field"]')

                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                for tr in list_tr:
                    make = tr.xpath('./td[2]/text()')[0].strip()
                    model = tr.xpath('./td[3]/text()')[0].strip()
                    year_start = int(tr.xpath('./td[1]/text()')[0].split('-')[0].strip())
                    year_end = int(tr.xpath('./td[1]/text()')[0].split('-')[-1].strip())

                    if make not in dict_vehicle:
                        dict_vehicle[make] = {model: [year_start, year_end]}
                    else:
                        if model not in dict_vehicle[make]:
                            dict_vehicle[make][model] = [year_start, year_end]
                        else:
                            dict_vehicle[make][model].append(year_start)
                            dict_vehicle[make][model].append(year_end)

                    dict_vehicle[make][model] = list(set(dict_vehicle[make][model]))
                    dict_vehicle[make][model].sort()
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        year_max = max(dict_vehicle[make][model])
                        year_min = min(dict_vehicle[make][model])
                        if year_max == year_min:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                        else:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                                
                oe = ';'.join(sorted(list(set([oe.strip() for oe in html.xpath('//table[@id="interchange-codes-table"]/tbody/tr/td[2]/text()')]))))

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Product_Id': input_.loc[a, 'Product_Id'],
                                         'Vehicle': vehicle,
                                         'OE': oe}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Product_Id': input_.loc[a, 'Product_Id'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Product_Id'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Product_Id'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Product_Id'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：500

[ok] - 1152841
[尝试次数：1] - [剩余数量：490] - [当前时间：16:34:21]

[ok] - 1152833
[尝试次数：1] - [剩余数量：489] - [当前时间：16:34:21]

[ok] - 1176157
[尝试次数：1] - [剩余数量：488] - [当前时间：16:34:21]

[ok] - 1188171
[尝试次数：1] - [剩余数量：487] - [当前时间：16:34:21]

[ok] - 1152832
[尝试次数：1] - [剩余数量：486] - [当前时间：16:34:21]

[ok] - 1152814
[尝试次数：1] - [剩余数量：485] - [当前时间：16:34:21]

[ok] - 1176156
[尝试次数：1] - [剩余数量：484] - [当前时间：16:34:21]

[ok] - 1188545
[尝试次数：1] - [剩余数量：483] - [当前时间：16:34:21]

[ok] - 1161722
[尝试次数：1] - [剩余数量：482] - [当前时间：16:34:22]

[ok] - 1161803
[尝试次数：1] - [剩余数量：481] - [当前时间：16:34:22]

[ok] - 1161786
[尝试次数：1] - [剩余数量：480] - [当前时间：16:34:23]

[ok] - 1188405
[尝试次数：1] - [剩余数量：479] - [当前时间：16:34:23]

[ok] - 1161852
[尝试次数：1] - [剩余数量：478] - [当前时间：16:34:23]

[ok] - 1176393
[尝试次数：1] - [剩余数量：477] - [当前时间：16:34:23]

[ok] - 1188153
[尝试次数：1] - [剩余数量：476] - [当前时间：16:34:23]

[ok] - 1161801
[尝试次数：1] - [剩余数量：475] - [当前时间：16:34:23]

[ok] - 1169650
[尝试次数：1] - [剩余数量：474] - [当前时间：16:34:23]

[ok] - 1176372
[尝试次数：1] - [剩余数量：473] - 

[ok] - 1209134
[尝试次数：2] - [剩余数量：344] - [当前时间：16:35:28]

[ok] - 1188157
[尝试次数：1] - [剩余数量：343] - [当前时间：16:35:29]

[ok] - 1161928
[尝试次数：3] - [剩余数量：342] - [当前时间：16:35:29]

[ok] - 1176392
[尝试次数：1] - [剩余数量：341] - [当前时间：16:35:29]

[ok] - 1188523
[尝试次数：1] - [剩余数量：340] - [当前时间：16:35:30]

[ok] - 1188400
[尝试次数：2] - [剩余数量：339] - [当前时间：16:35:30]

[ok] - 1103329
[尝试次数：1] - [剩余数量：338] - [当前时间：16:35:31]

[ok] - 1121177
[尝试次数：1] - [剩余数量：337] - [当前时间：16:35:31]

[ok] - 1081412
[尝试次数：1] - [剩余数量：336] - [当前时间：16:35:31]

[ok] - 1171080
[尝试次数：1] - [剩余数量：335] - [当前时间：16:35:31]

[ok] - 1209108
[尝试次数：4] - [剩余数量：334] - [当前时间：16:35:31]

[ok] - 1161787
[尝试次数：1] - [剩余数量：333] - [当前时间：16:35:32]

[ok] - 1162033
[尝试次数：1] - [剩余数量：332] - [当前时间：16:35:32]

[ok] - 1176509
[尝试次数：1] - [剩余数量：331] - [当前时间：16:35:33]

[ok] - 1176183
[尝试次数：2] - [剩余数量：330] - [当前时间：16:35:33]

[ok] - 1161720
[尝试次数：1] - [剩余数量：329] - [当前时间：16:35:33]

[ok] - 1178187
[尝试次数：1] - [剩余数量：328] - [当前时间：16:35:34]

[ok] - 1178225
[尝试次数：1] - [剩余数量：327] - [当前时间：16:

[ok] - 1162028
[尝试次数：8] - [剩余数量：197] - [当前时间：16:36:28]

[ok] - 1188395
[尝试次数：2] - [剩余数量：196] - [当前时间：16:36:28]

[ok] - 1188543
[尝试次数：3] - [剩余数量：195] - [当前时间：16:36:29]

[ok] - 1178216
[尝试次数：6] - [剩余数量：194] - [当前时间：16:36:29]

[ok] - 1188562
[尝试次数：2] - [剩余数量：193] - [当前时间：16:36:30]

[ok] - 1188552
[尝试次数：1] - [剩余数量：192] - [当前时间：16:36:30]

[ok] - 1188294
[尝试次数：1] - [剩余数量：191] - [当前时间：16:36:30]

[ok] - 1188167
[尝试次数：2] - [剩余数量：190] - [当前时间：16:36:31]

[ok] - 1188179
[尝试次数：2] - [剩余数量：189] - [当前时间：16:36:32]

[ok] - 1188402
[尝试次数：13] - [剩余数量：188] - [当前时间：16:36:32]

[ok] - 1211620
[尝试次数：3] - [剩余数量：187] - [当前时间：16:36:33]

[ok] - 1209022
[尝试次数：1] - [剩余数量：186] - [当前时间：16:36:34]

[ok] - 1188174
[尝试次数：3] - [剩余数量：185] - [当前时间：16:36:34]

[ok] - 1209091
[尝试次数：12] - [剩余数量：184] - [当前时间：16:36:35]

[ok] - 1216177
[尝试次数：7] - [剩余数量：183] - [当前时间：16:36:35]

[ok] - 1209090
[尝试次数：3] - [剩余数量：182] - [当前时间：16:36:35]

[ok] - 1209120
[尝试次数：1] - [剩余数量：181] - [当前时间：16:36:36]

[ok] - 1188279
[尝试次数：9] - [剩余数量：180] - [当前时间：1

[ok] - 1177976
[尝试次数：4] - [剩余数量：49] - [当前时间：16:37:27]

[ok] - 1192103
[尝试次数：3] - [剩余数量：48] - [当前时间：16:37:27]

[ok] - 1209159
[尝试次数：5] - [剩余数量：47] - [当前时间：16:37:27]

[ok] - 1121054
[尝试次数：4] - [剩余数量：46] - [当前时间：16:37:28]

[ok] - 1131675
[尝试次数：6] - [剩余数量：45] - [当前时间：16:37:28]

[ok] - 1140867
[尝试次数：1] - [剩余数量：44] - [当前时间：16:37:28]

[ok] - 1194184
[尝试次数：2] - [剩余数量：43] - [当前时间：16:37:28]

[ok] - 1140872
[尝试次数：2] - [剩余数量：42] - [当前时间：16:37:28]

[ok] - 1097914
[尝试次数：6] - [剩余数量：41] - [当前时间：16:37:29]

[ok] - 1140885
[尝试次数：1] - [剩余数量：40] - [当前时间：16:37:29]

[ok] - 1140899
[尝试次数：1] - [剩余数量：39] - [当前时间：16:37:29]

[ok] - 1140859
[尝试次数：1] - [剩余数量：38] - [当前时间：16:37:30]

[ok] - 1140836
[尝试次数：2] - [剩余数量：37] - [当前时间：16:37:30]

[ok] - 1140853
[尝试次数：2] - [剩余数量：36] - [当前时间：16:37:30]

[ok] - 1140873
[尝试次数：7] - [剩余数量：35] - [当前时间：16:37:30]

[ok] - 1161924
[尝试次数：1] - [剩余数量：34] - [当前时间：16:37:31]

[ok] - 1161931
[尝试次数：4] - [剩余数量：33] - [当前时间：16:37:33]

[ok] - 1194211
[尝试次数：3] - [剩余数量：32] - [当前时间：16:37:33]

[ok] - 116